<a href="https://colab.research.google.com/github/datasith/ML-Notebooks-TensorFlow/blob/main/Intro_Computational_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Computational Graphs with TensorFlow

In this notebook I provide a short introduction and overview of computational graphs using TensorFlow inspired by the PyTorch equivalent written by [Elvis Saravia](https://github.com/dair-ai/ML-Notebooks) et al.

There are several materials online that cover theory on the topic of computational graphs. However, it seems much easier to learn the concept using code, thus this is an attempt to bridge the gap that should be particularly useful for those on getting started with the subject.  

Inspired by Olah's article ["Calculus on Computational Graphs: Backpropagation"](https://colah.github.io/posts/2015-08-Backprop/), Elvis put together a few code snippets to get you started with computationsl graphs with PyTorch. Similarly, this TensorFlow companion notebook should complement that article, which you are encouraged to reference for more comprehensive explanations.

---
**Author:** Cisco Zabala ([@datasith](https://twitter.com/datasith) | [LinkedIn](https://www.linkedin.com/in/datasith/) | [Kaggle](https://kaggle.com/thedatasith) | [GitHub](https://github.com/datasith))

*Based on the work by Elvis Saravia ([Twitter](https://twitter.com/omarsar0) | [LinkedIn](https://www.linkedin.com/in/omarsar/)) on GitHub: [ML Notebooks](https://github.com/dair-ai/ML-Notebooks)*

## Why Computational Graphs?

When talking about neural networks in any context, [backpropagation](https://en.wikipedia.org/wiki/Backpropagation) is an important topic to understand because it is the algorithm used for training deep neural networks. 

Backpropagation is used to calculate derivatives which is what you need to keep optimizing parameters of the model and allowing the model to learn on the task at hand. 

Many of the deep learning frameworks today like TensorFlow perform backpropagation out-of-the-box using [**automatic differentiation**](https://www.tensorflow.org/guide/autodiff). 

To better understand how this is done it's important to talk about **computational graphs** which defines the flow of computations that are carried out throughout the network. Along the way we will use the `tensorflow.GradientTape` API to demonstrate in code how this works.

## Getting Started

Inspired by Olah's article on computational graphs, let's look at the following expression $e = (a + b) * (b+1)$. It helps to break it down to the following operations:

$$
\begin{aligned}&c=a+b \\&d=b+1 \\&e=c * d\end{aligned}
$$

This is not a neural network of any sort. We are just going through a very simple example of a chain of operations which you can be represented with computational graphs. 

Let's visualize these operations using a computational graph. Computational graphs contain **nodes** which can represent and input (tensor, matrix, vector, scalar) or **operation** that can be the input to another node. The nodes are connected by **edges**, which represent a function argument, they are pointers to nodes. Note that the computation graphs are directed and acyclic. The computational graph for our example looks as follows:

![](https://raw.githubusercontent.com/datasith/ML-Notebooks-TensorFlow/main/img/Intro_Computational_Graphs/tree-def.png)

*Source: Christopher Olah (2015)*

We can evaluate the expression by setting our input variables as follows: $a=2$ and $b=1$. This will allow us to compute nodes up through the graph as shown in the computational graph above.  

Rather than doing this by hand, we can use the [AutoGraph](https://www.tensorflow.org/api_docs/python/tf/autograph) engine provided by TensorFlow.

Let's import TensorFlow first:

In [9]:
import tensorflow as tf

Then, let's define the input tensors as placeholders for the input values; these will be filled with real data when we execute the graph:

In [10]:
  a = tf.Variable([2.])
  b = tf.Variable([1.])

## Static vs. Dynamic Computational Graphs

TensorFlow and PyTorch handle computational graphs differently and by extension, the calculation of gradients. Initially, TensorFlow graphs were static by default (this changed in TensorFlow 2) whereas PyTorch ones were always dynamic.

For static graphs, we create and connect all the variables at the beginning, and initialize them into a static (unchanging) session. This session and graph persists and is reused: it is not rebuilt after each iteration of training, making it efficient. However, with a static graph, variable sizes have to be defined at the beginning, which can be non-convenient for some applications, such as NLP with variable length inputs.

For dynamic graphs, we build them dynamically, immediately after we declare variables. This graph is thus rebuilt after each iteration of training. This allows for the flexibility to modify and inspect the internals of the graph at any time. However, they can take time to rebuild.

**References**: 

* https://cs230.stanford.edu/section/5/

---

With the input tensors defined, let's set up the computational graph. In TensorFlow 2.0, AutoGraph is automatically applied when using `tf.function`. AutoGraph transforms a subset of Python which operates on TensorFlow objects into equivalent TensorFlow graph code:

In [11]:
@tf.function
def create_graph(a, b, get_grads):
  with tf.GradientTape(persistent=True) as tape:
    c = a + b
    d = b + 1
    e = c * d 
  if get_grads == False:
    return {'out':e}
  else:
    de_dc = tape.gradient(e, c)
    de_dd = tape.gradient(e, d)
    de_de = tape.gradient(e, e)    
    return {'out' :e,
            'd_dc':de_dc,
            'd_dd':de_dd, 
            'd_de':de_de}
graph = tf.function(create_graph)

Note that we used `.GradientTape(persistent=True)` to "record" the operations executed inside its context onto a "tape". Later, we will use the TensorFlow tape to compute the gradients using [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

Now that we have our computational graph, we can check the result when evaluating the expression:

In [12]:
print(graph(a, b, False)['out'])

tf.Tensor([6.], shape=(1,), dtype=float32)


The output is a tensor with the value of `6.`, which verifies the results here: 

![](https://raw.githubusercontent.com/datasith/ML-Notebooks-TensorFlow/main/img/Intro_Computational_Graphs/tree-eval.png)
*Source: Christopher Olah (2015)*

## Derivatives on Computational Graphs

Using the concept of computational graphs we are now interested in evaluating the **partial derivatives** of the edges of the graph. This will help in gathering the gradients of the graph. Remember that gradients are what we use to train the neural network and those calculations can be taken care of by the automatic differentation engine. 

The intuition is: we want to know, for example, to what extent is the output $e$ affected by $c$ or $a$. In other words, if we change $a$ or $c$ a little, how does $e$ change. This is referred to as the partial derivative of $e$ with respect to $a$.

Note that for the variables we pass into the function, we can calculate the partial derivatives *externally* to the graph definition function. So for computing $\frac{\partial e}{\partial a}$, we can do:

In [13]:
with tf.GradientTape(persistent=True) as tape:
  out = graph(a,b, True)
  e = out['out']

de_da = tape.gradient(e, a)
de_da

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

It is important to understand the intuition behind this. Olah puts it best:

>Let’s consider how $e$ is affected by $a$. If we change $a$ at a speed of 1, $c$ also changes at a speed of $1$. In turn, $c$ changing at a speed of $1$ causes $e$ to change at a speed of $2$. So $e$ changes at a rate of $1*2$ with respect to $a$.


In other words, by hand this would be:

$$
\frac{\partial e}{\partial \boldsymbol{a}}=\frac{\partial e}{\partial \boldsymbol{c}} \frac{\partial \boldsymbol{c}}{\partial \boldsymbol{a}} = 2 * 1
$$

You can verify that this is correct by checking the manual calculations by Olah. Since $a$ is not directly connectected to $e$, we can use some special rule which allows to sum over all paths from one node to the other in the computational graph and mulitplying the derivatives on each edge of the path together.

![](https://raw.githubusercontent.com/datasith/ML-Notebooks-TensorFlow/main/img/Intro_Computational_Graphs/tree-eval-derivs.png)
*Source: Christopher Olah (2015)*

To check that this holds, let look at another example. How about caluclating the derivative of $e$ with respect to $b$, i.e., $\frac{\partial e}{\partial b}$?

We can get that through `b.grad`:

In [14]:
de_db = tape.gradient(e, b)
de_db

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>

If you work it out by hand, you are basically doing the following:

$$
\frac{\partial e}{\partial b}=1 * 2+1 * 3
$$

It indicates how $b$ affects $e$ through $c$ and $d$. We are essentially summing over paths in the computational graph.

If we are interested in examining the derivative of $e$ with respect to $c$—or any other internal variable—how do we obtain this? In other words, for examining the value of $\frac{\partial e}{\partial c}$ in this case with the given $a$ and $b$, we can include the calculation as part of our computational graph definition (also using `tf.GradientTape`), and return the calculations as part of our output:

In [17]:
int_grads = {
             'd_dc' : out['d_dc'],
             'd_dd' : out['d_dd'],
             'd_de' : out['d_de'],
            }             
for grad in int_grads:
  print(int_grads[grad])

tf.Tensor([2.], shape=(1,), dtype=float32)
tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor([1.], shape=(1,), dtype=float32)


Here are all the gradients collected, including non-leaf nodes:

In [16]:
print(de_da)
print(de_db)
for grad in int_grads:
  print(int_grads[grad])

tf.Tensor([2.], shape=(1,), dtype=float32)
tf.Tensor([5.], shape=(1,), dtype=float32)
tf.Tensor([2.], shape=(1,), dtype=float32)
tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor([1.], shape=(1,), dtype=float32)


You can use the computational graph above to verify that everything is correct. This is the power of computational graphs and how they are used by automatic differentation engines. It's also a very useful concept to understand when developing neural networks architectures and their correctness.

## Next Steps

In this notebook, I've provided a simple and intuitive explanation to the concept of computational graphs using TensorFlow. I highly recommend to go through [Olah's article](https://colah.github.io/posts/2015-08-Backprop/) for more on the topic, and also comparing this notebook with its counterpart [implemented in PyTorch](https://colab.research.google.com/drive/1eG1AF36Wa0EaANandAhrsbC3j04487SH?usp=sharing#scrollTo=sbJvhj5m13Zq).

In the next tutorial, I will be applying the concept of computational graphs to more advanced operations you typically see in a neural network. In fact, if you are interested in this, and you are feeling comfortable with the topic now, you can check out these TensorFlow tutorials:

- [Better performance with `tf.function`](https://www.tensorflow.org/guide/function)
- [Introduction to gradients and automatic differentiation (TensorFlow)](https://www.tensorflow.org/guide/autodiff)

And here are some other useful references used to put this article together:

- [Hacker's guide to Neural Networks
](http://karpathy.github.io/neuralnets/)
- [Backpropagation calculus](https://www.youtube.com/watch?v=tIeHLnjs5U8&ab_channel=3Blue1Brown)

